# heatmap radial average vs pp_delay

In [1]:
%pylab qt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np  
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import os
import src.pyabel_polar
import scipy

# Utility functions

In [3]:
def number_of_trains_from_hdf(hdf_file_complete_path):
    '''Retrun number of recorded FEL trains in HDF file'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        trains = len(h_file['tpx3Times/triggerNr'][:])
    print('Number of trains in HDF file: {:.2e}'.format(trains))
    return trains

def data_from_hdf(hdf_file_complete_path, event_type = 'raw'):
    '''Read data from TimePix HDF files
    Choose raw or centroided data, default is centroided data'''
    with h5py.File(hdf_file_complete_path, 'r') as h_file:
        tof = h_file[str(event_type)+'/tof'][:]
        x_pos = h_file[str(event_type)+'/x'][:]
        y_pos = h_file[str(event_type)+'/y'][:]
#     print('Reading from HDF5 - number of events: {:.2e}'.format(len(tof)))
    return tof, x_pos, y_pos

def data_sliced_by_tof(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, event_type = 'raw'):
    '''Slice data with respect to time-of-flight dimension'''
    tof, x_pos, y_pos = data_from_hdf(hdf_file_complete_path, event_type)
    sliced_x_pos = x_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_y_pos = y_pos[np.logical_and(tof > tof_start, tof < tof_end)]
    sliced_tof = tof[np.logical_and(tof > tof_start, tof < tof_end)]
#     print('Slicing - number of events: {:.2e} | {:.2%}'.format(len(sliced_tof), len(sliced_tof)/len(tof)))
    return sliced_tof, sliced_x_pos, sliced_y_pos

def reduce_raw_data(tof, x_pos, y_pos, number_of_events):
    '''Reduce data for visualization'''
    return tof[:number_of_events], x_pos[:number_of_events],y_pos[:number_of_events]

def tof_conversion(tof, time_unit):
    '''Convert time axis'''
    if time_unit == None:
        return tof, 's'
    if time_unit == 'milli':
        return tof*10**3, 'ms'
    if time_unit == 'micro':
        return tof*10**6, 'us'

def plot_tof(tof, hist_bins=100, time_unit = None):     
    '''Plot time-of-flight spectrum via histogram'''
    fig = plt.subplots(num = 1)
    plt.clf()
    tof, time_tof_unit = tof_conversion(tof, time_unit)
    plt.hist(tof, bins = hist_bins)
    plt.title('histogram: time-of-flight')
    plt.xlabel('ToF [{}]'.format(time_tof_unit))
    plt.ylabel('number of events')
    plt.show()
    
def vmi_image(x_pos, y_pos, show_image = True):
    '''Display VMI image - cmax empirically found to surpress hot pixel '''
    fig = plt.figure(num = 6)
    plt.clf()
    counts, xbins, ybins, image = plt.hist2d(x_pos, y_pos,bins= (255, 255), cmax= 1000)
    plt.colorbar()
    plt.xlabel('x_pos [px]')
    plt.ylabel('y_pos [px]')
    if show_image == False:
        plt.close()
    return counts
    
def display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start = 0 , tof_end = 0.1, hist_bins = 100, time_unit = None, event_type = 'raw'):
    '''Display VMI Image and time-of-flight spetrum'''
    tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start, tof_end, event_type)
    plot_tof(tof, hist_bins, time_unit)
    vmi_image(x_pos,y_pos)

    
def transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius ):
    
    counts = vmi_image(x_pos, y_pos, show_image = False)
    image_cart = np.flipud(counts.transpose())
    
    image_polar, r_grid, theta_grid = src.pyabel_polar.reproject_image_into_polar(image_cart, origin=(x_center,y_center))
    radial_ave = np.sum(image_polar, axis=1)
    
    fig = plt.figure(figsize = (10,10),num = 6)
    plt.clf()
    plt.imshow(image_cart)
    plt.scatter(x_center, y_center, color='r')
    plt.gcf().gca().add_artist(plt.Circle((x_center, y_center), radius, color='r', fill=False))
    plt.xlabel('x_posr [px]')
    plt.ylabel('y_posr [px]')
    plt.title('VMI image')

#     fig = plt.figure(num = 7)
#     plt.clf()
#     plt.imshow(image_polar)
#     plt.title('Image - polar coordinates')

    fig = plt.figure(num = 8)
    plt.clf()
    plt.plot(radial_ave,'r-')
    plt.title('Radial average')
    plt.xlabel('r [px]')
    plt.ylabel('counts')
    plt.show()
    
    return radial_ave

### Define source file & describe measurment

In [ ]:
run_number = 69

hdf_file_path = '/gpfs/bl1/current/processed/timepix_hdf/'
file_start = "run_"+str(run_number).zfill(4)
not_file_end = 'rawOnly.hdf5'
hdf_file = [i for i in os.listdir(hdf_file_path) if os.path.isfile(os.path.join(hdf_file_path,i)) and i.startswith(file_start) and not i.endswith(not_file_end)][0]
hdf_file_complete_path = hdf_file_path+hdf_file
assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'

### Complete time-of-flight

In [ ]:
tof_start = 0E-6
tof_end = 12E-6
tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end)

plot_tof(tof, hist_bins = 500, time_unit = 'micro')

### Slice time-of-flight dimension and create VMI image

In [ ]:
tof_start = 7.55E-6
tof_end = 8.01E-6

display_tof_and_vmi_of_tof_interval(hdf_file_complete_path, tof_start, tof_end, event_type = 'raw',time_unit = 'micro')

### Chose center and calc radial average

In [ ]:
x_center, y_center = 130, 120        #I2+
radius = 18

# x_center, y_center = 131, 127          #I6+
# radius = 18

tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end, event_type="raw")
radial_average = transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius)

### Save to intermediate results 

In [ ]:
def save_all_delay_for_fragment(file_prefix ,run_number, delay_stage, tof_start,tof_end, x_center, y_center, radius):
    
    print(run_number)
    hdf_file_path = '/gpfs/bl1/current/processed/timepix_hdf/'
    file_start = "run_"+str(run_number).zfill(4)
    not_file_end = 'rawOnly.hdf5'
    print()
    hdf_file = [i for i in os.listdir(hdf_file_path) if os.path.isfile(os.path.join(hdf_file_path,i)) and i.startswith(file_start) and not i.endswith(not_file_end)][0]
    hdf_file_complete_path = hdf_file_path+hdf_file
    assert os.path.isfile(hdf_file_complete_path), 'File does not exist!'
    
    tof, x_pos, y_pos = data_sliced_by_tof(hdf_file_complete_path, tof_start , tof_end, event_type = "raw")
    radial_average = transform_vmi_to_polar(x_pos, y_pos, x_center, y_center, radius) 
    number_of_trains = number_of_trains_from_hdf(hdf_file_complete_path)
    
    save_to_path = 'intermediate_data/'
    save_file_name = str(file_prefix)+"_"+str(fragment)+"_"+str(delay_stage)
    print(str(file_prefix)+"_"+str(fragment)+"_"+str(delay_stage))
    np.savez(save_to_path+save_file_name, radial_average=radial_average, fragment= fragment, delay_stage = delay_stage, number_of_trains = number_of_trains)
    

In [ ]:
save_to_path = 'intermediate_data/'

file_prefix = 'raw_test'
fragment = 'I6+'

run_number_list = [52,53,54,55,56,59,60,61,62,63,65,66,67,68,69,70]
delay_stage_pos_list = [64,63,62,61,60,61.5,60.5,62.5,63.5,59.5,62.25,62.75,61.75,61.25,60.75,60.25]

# run_number_list = [63,69]
# delay_stage_pos_list = [59.5,60.75]

assert len(run_number_list) == len(delay_stage_pos_list), 'File does not exist!'

In [ ]:
for i in range(len(run_number_list)):
    save_all_delay_for_fragment(file_prefix, run_number_list[i], delay_stage_pos_list[i], tof_start,tof_end, x_center, y_center, radius)

___
# Create heatmap

In [4]:
def simple_plots(radial_averages_norm, delay_stage_pos, fragment):
    
    colors = cm.rainbow(np.linspace(0, 1, len(radial_averages_norm)))
    print(colors.shape)
    print(colors[0])
    
    fig = plt.figure(figsize = (10,10),num = 10)
    for i in range(len(radial_averages_norm)):
        plt.plot(radial_averages_norm[i],label=delay_stage_pos[i], color = colors[i])
    plt.xlabel('r')
    plt.ylabel('counts')
    plt.legend()
    plt.title(str(fragment))
    
def heatmap(radial_averages_norm, fragment):
    fig = plt.figure(figsize = (10,10),num = 11)
    image = np.column_stack(radial_averages_norm)
    plt.imshow(image, aspect="auto", origin="lower")
    plt.xticks(np.arange(len(delay_stage_pos)), delay_stage_pos)
    print('Number of delay steps:',len(delay_stage_pos))
    plt.title(str(fragment))
    plt.show()
    
    
def load_radial_data(file_prefix,fragment):
    
    load_from_path = 'intermediate_data/'
    files = [i for i in os.listdir(load_from_path) if os.path.isfile(os.path.join(load_from_path,i)) and str(file_prefix)+"_"+str(fragment) in i]
    
    radial_averages =[]
    delay_stage_pos = []
    norm_counts = []
    
    for i in files:
        npzfile = np.load(load_from_path+i)
        radial_averages.append(npzfile['radial_average'])
        delay_stage_pos.append(float(npzfile['delay_stage']))
        fragment = (npzfile['fragment'])
        norm_counts.append(float(npzfile['number_of_trains']))
        
    radial_averages = np.array(radial_averages)
    delay_stage_pos = np.array(delay_stage_pos)
    norm_counts = np.array(norm_counts)

    sort_ind = np.argsort(delay_stage_pos)
    radial_averages = radial_averages[sort_ind]
    norm_counts = norm_counts[sort_ind]
    delay_stage_pos = delay_stage_pos[sort_ind]
    
    radial_averages_norm = []
    for i in range(len(radial_averages)):
        radial_averages_norm.append(radial_averages[i]/norm_counts[i])
    radial_averages_norm = np.array(radial_averages_norm)

    print('fragment:',fragment) 
    print('number of delay_stage_pos:',len(delay_stage_pos))
    print('delay_stage_steps:',delay_stage_pos)

    return radial_averages_norm, delay_stage_pos     
    

### Re-read data

In [5]:
file_prefix = 'raw_test'
fragment = 'I6+'

radial_averages_norm, delay_stage_pos = load_radial_data(file_prefix,fragment)

fragment: I6+
number of delay_stage_pos: 16
delay_stage_steps: [59.5  60.   60.25 60.5  60.75 61.   61.25 61.5  61.75 62.   62.25 62.5
 62.75 63.   63.5  64.  ]


### simple plots

In [9]:
simple_plots(radial_averages_norm, delay_stage_pos, fragment)

(16, 4)
[0.5 0.  1.  1. ]


### heatmap

In [7]:
heatmap(radial_averages_norm, fragment)

Number of delay steps: 16
